# Transfer Learning

There are various models available which can be used for transfer learning
- VGG16
- VGG19
- ResNet varients etc

### Resnet speciality
Resnet has a special network or connection called skip connection. Which basically adds the previous(before skip connection) weights to current(after skip connection) weight ... 

![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.2YRpCsirP4qRHrJG8WJXtQAAAA%26pid%3DApi&f=1)

we can implement this as

In [3]:
import torch.nn as nn

class ResNetBlock(nn.Module):
  def __init__(self, n_inp, n_op, kernel_size, stride=1):
    super(ResNetBlock, self).__init__()
    padding = kernel_size - 2
    self.conv = nn.Sequential(
        nn.Conv2d(n_inp, n_op, kernel_size, stride,
                  padding=padding),
                  nn.BatchNorm2d(),
                  nn.ReLU()
                  )
  
    def forward(self, x):
      x = self.conv(x) + x
      return x



In [5]:
import torch
from torchvision import models

device = 'cpu'

def get_model():
  model = models.resnet18(pretrained=True)
  for param in model.parameters():
    param.requires_grad = False # freezing the layers

    # changing the required segment 
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1)) 
    model.fc = nn.Sequential(nn.Flatten(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())

    # Loss and optimizer
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr= 1e-3)
  return model.to(device), loss_fn, optimizer